# 1. Set Up

In [ ]:
# !spack find -lvp tfel
# !echo "$(spack find -p tfel | awk '/\/mofem_install\// {print $NF "/lib"}')"
# %env LD_LIBRARY_PATH=/mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-mvfpqw7u4c23su7hj7g4leuwmykrjmcx/lib

In [ ]:
import math
import os
import re
import sys
import time
import json
from pathlib import Path
import subprocess
import zipfile
import pydantic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyvista as pv
import gmsh

sys.path.append('/mofem_install/jupyter/thomas/mfront_example_test/src')

import setup
import core
import custom_models as cm
import utils as ut
import plotting
import calculations as calc
import postprocess
    
import matplotlib
matplotlib.rc('figure', figsize=(7, 7))

# 2. Simulation Parameters

In [ ]:
#in MPa
def initialize_parameters(custom_mesh_filepath: str = None, soil_model: cm.PropertyTypeEnum = cm.PropertyTypeEnum.le, **kwargs) -> cm.AttrDict:
    params = cm.AttrDict()
    setattr(params, "custom_mesh_filepath", custom_mesh_filepath)
    setattr(params, "soil_model", soil_model)
    for key,value in kwargs.items():
        setattr(params, key, value)
    params.interface = False
    params.pile_manager = cm.PileManager(x=0, y=0, z=0, dx=0, dy=0, R=1, r=0.975,
                                         stickup_height = 10,
                                         embedded_depth = -10.5,
                                preferred_model= cm.PropertyTypeEnum.le,
                                props = {
                                    cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=200000, poisson_ratio=0.3),
                                },
                                interface = params.interface,
                                )

    soil_layer_1 = cm.SoilLayer(
        depth = -2,
        preferred_model= params.soil_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=96, poisson_ratio=0.49),
            cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=96, poisson_ratio=0.49, HardeningSlope = 0.9, YieldStress = 0.28),
            cm.PropertyTypeEnum.dp: cm.DruckerPragerProperties(youngs_modulus=96, poisson_ratio=0.3, phi=np.radians(15), c=2, v=np.radians(15)),
            cm.PropertyTypeEnum.dpHYPER: cm.DPHYPERProps(youngs_modulus=96, poisson_ratio=0.3, phi=np.radians(15), c=2, v=np.radians(15), proximity = 1e-6),
            cm.PropertyTypeEnum.mcc: cm.CamClayProperties(),
            }, 
            )
    soil_layer_2 = cm.SoilLayer(
        depth = -1.4,
        preferred_model= params.soil_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=182.1, poisson_ratio=0.49),
            cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=182.1, poisson_ratio=0.49, HardeningSlope = 1.8, YieldStress = 0.28),
            cm.PropertyTypeEnum.dp: cm.DruckerPragerProperties(youngs_modulus=182.1, poisson_ratio=0.3, phi=np.radians(15), c=2, v=np.radians(15)),
            cm.PropertyTypeEnum.dpHYPER: cm.DPHYPERProps(youngs_modulus=182.1, poisson_ratio=0.3, phi=np.radians(15), c=2, v=np.radians(15), proximity = 1e-6),
            cm.PropertyTypeEnum.mcc: cm.CamClayProperties(),
            }, 
            )
    soil_layer_3 = cm.SoilLayer(
        depth = -7.1,
        preferred_model= params.soil_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=351.3, poisson_ratio=0.49),
            cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=351.3, poisson_ratio=0.49, HardeningSlope = 3.5, YieldStress = 0.28),
            cm.PropertyTypeEnum.dp: cm.DruckerPragerProperties(youngs_modulus=351.3, poisson_ratio=0.3, phi=np.radians(15), c=2, v=np.radians(15)),
            cm.PropertyTypeEnum.dpHYPER: cm.DPHYPERProps(youngs_modulus=351.3, poisson_ratio=0.3, phi=np.radians(15), c=2, v=np.radians(15), proximity = 1),
            cm.PropertyTypeEnum.mcc: cm.CamClayProperties(),
                 }, 
            )
    soil_layer_4 = cm.SoilLayer(
        depth = -29.5,
        preferred_model= params.soil_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=668.4, poisson_ratio=0.49),
            cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=668.4, poisson_ratio=0.49, HardeningSlope = 6.7, YieldStress = 0.28),
            cm.PropertyTypeEnum.dp: cm.DruckerPragerProperties(youngs_modulus=668.4, poisson_ratio=0.3, phi=np.radians(15), c=2, v=np.radians(15)),
            cm.PropertyTypeEnum.dpHYPER: cm.DPHYPERProps(youngs_modulus=668.4, poisson_ratio=0.3, phi=np.radians(15), c=2, v=np.radians(15), proximity = 1e-6),
            cm.PropertyTypeEnum.mcc: cm.CamClayProperties(),
            }, 
            )
    
    # params.interface_manager = cm.InterfaceManager(
    #     preferred_model = cm.PropertyTypeEnum.vM,
    #     props = {
    #         cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=96, poisson_ratio=0.3),
    #         cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=96, poisson_ratio=0.49,HardeningSlope = 5000, YieldStress = 20),
    #         cm.PropertyTypeEnum.dp: cm.DruckerPragerProperties(youngs_modulus=668.4, poisson_ratio=0.49, phi=np.radians(0.05), c=0, v=np.radians(0.05)),
    #         },
    #         )
    
    #=======================================================
    #=======================================================
    #=======================================================
    #if the soil domain is a cuboid
    params.box_manager = cm.BoxManager(x=-80, y=-80, z=0, dx=160, dy=80,
        layers=[
        soil_layer_1,
        soil_layer_2,
        soil_layer_3,
        soil_layer_4,   
        ],
        far_field_size=5,
        near_field_dist=5,
        near_field_size=0.05,
    )
    #if the soil domain is a half cylinder
    params.cylinder_manager = cm.CylinderManager(
        x=0, 
        y=0, 
        z=0, 
        dx=0, 
        dy=0, 
        r=80,
        pile_R = params.pile_manager.R,
        pile_r = params.pile_manager.r,
        layers=[
        soil_layer_1,
        soil_layer_2,
        soil_layer_3,
        soil_layer_4,   
        ],
        mesh_radial_progression = 1.3,
        mesh_radial_divisions = 10,
    )
    #=======================================================
    #=======================================================
    #=======================================================
    
    def custom_generate_physical_groups(params) -> list[cm.PhysicalGroup]:
        physical_group_list = [
            cm.PhysicalGroup(
                meshnet_id=1,
                name="MFRONT_MAT_1",
                preferred_model=params.cylinder_manager.layers[0].preferred_model,
                props=params.cylinder_manager.layers[0].props,
                ),
            cm.PhysicalGroup(
                meshnet_id=2,
                name="MFRONT_MAT_2",
                preferred_model=params.cylinder_manager.layers[1].preferred_model,
                props=params.cylinder_manager.layers[1].props,
                ),
            cm.PhysicalGroup(
                meshnet_id=3,
                name="MFRONT_MAT_3",
                preferred_model=params.cylinder_manager.layers[2].preferred_model,
                props=params.cylinder_manager.layers[2].props,
                ),
            cm.PhysicalGroup(
                meshnet_id=4,
                name="MFRONT_MAT_4",
                preferred_model=params.cylinder_manager.layers[3].preferred_model,
                props=params.cylinder_manager.layers[3].props,
                ),
            cm.PhysicalGroup(
                meshnet_id=5,
                name="MFRONT_MAT_5",
                preferred_model=params.pile_manager.preferred_model,
                props=params.pile_manager.props,
                ),
        ]
        return physical_group_list
    
    if getattr(params, "custom_mesh_filepath", False):
        try:
            params.custom_generate_physical_groups = custom_generate_physical_groups
        except NameError as e:
            raise NameError("Must provided the custom callback 'custom_generate_physical_groups' to inject attributes to the main commmand.")
   
    params.nproc = 8 # number of processors/cores used
    params.dim = 3
    
    params.wk_dir = Path(f"/mofem_install/jupyter/thomas/mfront_example_test")
    params.read_med_exe = "/mofem_install/jupyter/thomas/um_view/bin/read_med"
    params.h5m_to_vtk_converter = "/mofem_install/jupyter/thomas/um_view/bin/convert.py"
    params.partition_exe = "/mofem_install/jupyter/thomas/um_view/bin/mofem_part"
    params.exe = f"/mofem_install/jupyter/thomas/um_view/tutorials/adv-1/contact_3d"
    params.paraview_path = "/mofem_install/jupyter/thomas/ParaView-5.13.1-MPI-Linux-Python3.10-x86_64/bin/pvpython"
    params.ffmpeg_path = '/mofem_install/jupyter/thomas/ffmpeg-7.0.2-amd64-static/ffmpeg'
    params.preset_dir = '/mofem_install/jupyter/thomas/mfront_example_test/src/presets'
    
    return params

In [ ]:
params = initialize_parameters(
    order = 2, #order of approximation functions
    case_name = "pile",
    custom_mesh_filepath="/mofem_install/jupyter/thomas/mfront_example_test/user_data/pile_4.cub", 
    base = "hex",
    soil_model=cm.PropertyTypeEnum.vM,
    
    final_time = 1, # [s]
    time_step = 1 / 200, # [s]
    options_file = "/mofem_install/jupyter/thomas/mfront_example_test/param_file.petsc",
    
    save_gauss = 0,
    
    FEA_completed=False,
    # days_since_epoch=105,
    # sim_otd=30,
    
    # prescribed_force = cm.ForceBoundaryCondition(fx=0,fy=0,fz=0),
    prescribed_disp = cm.SurfaceBoundaryCondition(disp_ux=0.5, disp_uy=0 ,disp_uz=0),
    
    points_of_interest = [
        cm.Point(x=1,y=0,z=10),
        cm.Point(x=-1,y=0,z=0),
        cm.Point(x=1,y=0,z=0),
    ],
    line_of_interest = cm.Line(pt1=cm.Point(x=-1,y=0,z=10), pt2=cm.Point(x=-1,y=0,z=-40),),
    
) # 2. Simulation Parameters
params = setup.setup(params) # 3. Log paths and meta
if params.FEA_completed:
    # core.export_to_vtk(params)
    # postprocess.extract(params)
    # postprocess.calculate(params)
    # postprocess.plot(params)
    pass
else:
    core.generate_mesh(params) # 4. Generate the mesh
    core.mofem_compute(params) # 5. Running the analysis and export to .vtk file format
    core.export_to_vtk(params)
    postprocess.extract(params)
    postprocess.calculate(params)
    postprocess.plot(params)
    